# K-Nearest Neighbors Regressor from Scratch

Rules:
1. No dependencies
2. Only using base python functionality

#### The KNN Algorithm:
1. For each test observation, calculate distance from observation to each training observation
2. Sort neighbors and select nearest $k$
3. Use the target values associated with those neighbors
4. Return the mean from those values (regressor)

In [23]:
# Loading help packages!
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split


RANDOM_STATE = 666

In [24]:
# loading data!
X, y = load_boston(return_X_y = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = RANDOM_STATE)

### The setup - 4 main methods:
1. ~~`euclidean_distance` - calculates distance~~
2. ~~`k_nearest_neighbor_indices` - output a list of the indices for nearest neighbors~~
3. ~~`predict` - outputs the predictions~~
4. ~~`r2_score` - assess model's performance~~

### Euclidean distance

$$
\text{Euclidean distance} = \sqrt{\sum_{i=1}^{n}(a_i - b_i)^2}
$$

In [68]:
class KNNRegressor():
    def __init__(self, k = 5):
        self.k = k
    
    def __repr__(self):
        return f"KNNRegressor {{ k = {self.k} }}"
    
    def fit(self, X_train, y_train):
        self.X_train_ = X_train
        self.y_train_ = y_train
        
    def euclidean_distance(self, a, b):
        total = 0
        for a_i, b_i in zip(a, b):
            total += (a_i - b_i) ** 2
        return total ** .5
    
    def k_nearest_neighbor_indices(self, distances):
        # construct list of tuples (distance, index)
        distances_and_indices = [(distance, i) for i, distance in enumerate(distances)]
        
        # sort
        distances_and_indices.sort()
        
        # grab first k
        k_distances_and_indices = distances_and_indices[:self.k]
        
        return [i for _, i in k_distances_and_indices]
    
    def predict_row(self, X_row):
        # get distances from give observations and ALL trianing observations
        distances = [self.euclidean_distance(train_row, X_row) for train_row in self.X_train_]
        
        # get k-nearest indices using k_nearest_neighbor_indices
        k_indices = self.k_nearest_neighbor_indices(distances)
        
        # get k-nearest target values (self.y_train_)
        values = [self.y_train_[index] for index in k_indices]
        
        # return average
        return sum(values)/len(values)
    
    def predict(self, X):
        return [self.predict_row(X_row) for X_row in X]
    
    def r2_score(self, y_true, y_pred):
        # numerator
        ssr_model = 0
        for y_i, y_hat in zip(y_true, y_pred):
            ssr_model += (y_i - y_hat) ** 2
        
        # denominator
        y_true_mean = sum(y_true) / len(y_true)
        ssr_total = 0
        for y_i in y_true:
            ssr_total += (y_i - y_true_mean) ** 2
        
        try:
            return 1 - (ssr_model / ssr_total)
        except:
            return 0.0 # float
        
    def score(self, X, y_true):
        preds = self.predict(X)
        return self.r2_score(y_true, preds)

### $R^2$ score

$$
R^2 = 1 - \frac{SSR_{model}}{SSR_{total}} = 1 - \frac{\sum_{i = 1}^n (y_i - \hat{y})^2}{\sum_{i = 1}^n (y_i - \bar{y})^2}
$$

In [69]:
# instantiate
knn = KNNRegressor(k = 10)

# fit
knn.fit(X_train, y_train)

In [70]:
# train r^2 score
knn.score(X_train, y_train)

0.5971528837505538

In [71]:
# test r^2 score
knn.score(X_test, y_test)

0.381556121236996

### Compare to `sklearn`

In [72]:
from sklearn.neighbors import KNeighborsRegressor

In [73]:
sk_knn = KNeighborsRegressor(n_neighbors = 10, metric = "euclidean")

sk_knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='euclidean',
                    metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                    weights='uniform')

In [74]:
# train r^2 score
sk_knn.score(X_train, y_train)

0.597152883750554

In [75]:
# test r^2 score
sk_knn.score(X_test, y_test)

0.3815561212369959